In [18]:
import os
import pandas as pd
import numpy as np
import sklearn
import statsmodels.api as sm
import patsy
from patsy import dmatrices
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [26]:
merged = pd.read_csv("merged_with_gender.tsv", delimiter="\t")
# filter out examples where we failed to infer gender
genders = ['male', 'female']
df_filtered = merged[merged['gender'].isin(genders)]
# some cleaning
df_filtered["knowledge"] = pd.to_numeric(df_filtered["knowledge"].str.strip("&nbsp;"), errors='coerce')
# staff often missing
df_filtered["staff"] = pd.to_numeric(df_filtered["staff"].str.strip("&nbsp;"), errors='coerce')
df_filtered.shape

/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(53401, 11)

In [33]:
df_filtered.head()

,Unnamed: 0,dr-id,gendel,gender,helpful,insurance,knowledge,punctual,rating,review-text,staff
0,0,1,NaN,male,5,Insurance:Unknown,5,5,1,A CARING DOCTOR WHO CARES ABOUT HIS PATIENTS. ...,5
1,1,1,NaN,male,5,Insurance:Medicare,5,5,1,We have used Dr. Carden all of his career and ...,5
2,2,1,NaN,male,5,Insurance:Blue Cross / Blue Shield,5,5,1,"Very good doctor, caring. I think he knows e...",5
3,3,1,NaN,male,5,Insurance:United Healthcare,5,4,1,Dr. Carden is an outstanding physician. He spe...,4
4,4,1,NaN,male,5,Insurance:Unknown,5,5,1,Dr.Carden Is a great doctor that listens to hi...,5


In [22]:
grouped = df_filtered.groupby('dr-id')

Stolen from stanford people: https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt

In [28]:
stop_words = [w.strip() for w in open("stopwords.txt").readlines()]
stop_words[:10]

['!!', '?!', '??', '!?', '`', '``', "''", '-lrb-', '-rrb-', '-lsb-']

Let's remove gender pronouns since these aren't terribly interesting as predictors

In [29]:
gender_pronouns = ["she", "her", "hers", "his", "he", "guy", "she's", "he's", "commentshe", "commentsshe", "man", "woman"]
stop_words.extend(gender_pronouns)

Merge reviews for individual doctors (we have multiple reviews for each!)

In [30]:
unique_drs = df_filtered["dr-id"].unique()
unique_drs.shape
grouped = df_filtered.groupby('dr-id')
texts, sexes, helpful, knowledge, punctual, staff, ids = [], [], [], [], [], [], []
for i, dr in grouped:
    cur_text = " ".join(dr['review-text'].values)
    texts.append(cur_text)
    cur_sex = dr["gender"].values[0]

    helpful.append(dr["helpful"].mean(skipna=True)) # assuming missing at random!
    knowledge.append(dr["knowledge"].mean(skipna=True))
    punctual.append(dr["punctual"].mean(skipna=True))
    staff.append(dr["staff"].mean(skipna=True))

    ids.append(dr["dr-id"].values[0])
    sexes.append(cur_sex)

dr_frame = pd.DataFrame({"sex":sexes, "y_helpful":helpful, "y_knowledge":knowledge, "y_punctual":punctual, "y_staff":staff, "id":ids})

In [38]:
dr_frame.shape

(16488, 6)

How many females v males?

In [39]:
sexes.count("female")/float(len(sexes))
sexes.count("female")

(16469, 6)

So about 72% male

Note; should probably pull out and pool reviews for physicians -- here we treat individual reviews for the same physician as multiple / independent instances

## Regression analysis for gender-score 

**Important caveats** we are making several assumptions here, including:
* We are treating the means as a point estimates and not explicitly weighting by variance
* We are ignoring what are likely confounders, including specialties -- e.g., there may be certain specialties that receive lower scores and perhaps women are overrepresented in these. We have no way of knowing.

In [55]:
y_helpful, X = dmatrices('y_helpful ~ sex', data=dr_frame.dropna(subset=["y_helpful"]), return_type='dataframe')
mod = sm.OLS(y_helpful, X)
res_helpful = mod.fit()
print(res_helpful.summary())

                            OLS Regression Results                            
Dep. Variable:              y_helpful   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     13.99
Date:                Fri, 28 Oct 2016   Prob (F-statistic):           0.000185
Time:                        17:57:11   Log-Likelihood:                -28355.
No. Observations:               16485   AIC:                         5.671e+04
Df Residuals:                   16483   BIC:                         5.673e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       3.6467      0.020    184.220      

In [62]:
import coef_plots
import importlib
importlib.reload(coef_plots)
point_estimates = np.copy(res_helpful.params.values)
# add intercept to male factor
point_estimates[1] = point_estimates[0]+point_estimates[1]
lowers = point_estimates - 1.96*res_helpful.bse.values
uppers = point_estimates + 1.96*res_helpful.bse.values
names = ["female", "male"]
colors = ["black", "blue"]
coef_plots.point_ests_and_CIs(point_estimates, lowers, uppers, names, colors, "helpfulness.pdf", xtitle="Helpfulness" )

/anaconda/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [52]:
res_helpful.params.values

array([ 3.64665305,  0.0874156 ])

In [43]:
y_knowledge, X = dmatrices('y_knowledge ~ sex', data=dr_frame.dropna(subset=["y_knowledge"]), return_type='dataframe')
mod = sm.OLS(y_knowledge, X)
res_knowledge = mod.fit()
print(res_knowledge.summary())

                            OLS Regression Results                            
Dep. Variable:            y_knowledge   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     17.44
Date:                Fri, 28 Oct 2016   Prob (F-statistic):           2.98e-05
Time:                        17:03:37   Log-Likelihood:                -27077.
No. Observations:               16485   AIC:                         5.416e+04
Df Residuals:                   16483   BIC:                         5.417e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       3.8143      0.018    208.224      

In [63]:
import coef_plots
import importlib
importlib.reload(coef_plots)
point_estimates = np.copy(res_knowledge.params.values)
# add intercept to male factor
point_estimates[1] = point_estimates[0]+point_estimates[1]
lowers = point_estimates - 1.96*res_knowledge.bse.values
uppers = point_estimates + 1.96*res_knowledge.bse.values
names = ["female", "male"]
colors = ["black", "blue"]
coef_plots.point_ests_and_CIs(point_estimates, lowers, uppers, names, colors, "knowledge.pdf", xtitle="Knowledge" )

/anaconda/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [42]:
y_punctual, X = dmatrices('y_punctual ~ sex', data=dr_frame.dropna(subset=["y_punctual"]), return_type='dataframe')
mod = sm.OLS(y_punctual, X)
res_punc = mod.fit()
print(res_punc.summary())

                            OLS Regression Results                            
Dep. Variable:             y_punctual   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     41.23
Date:                Fri, 28 Oct 2016   Prob (F-statistic):           1.39e-10
Time:                        17:03:21   Log-Likelihood:                -26066.
No. Observations:               16469   AIC:                         5.214e+04
Df Residuals:                   16467   BIC:                         5.215e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       3.6203      0.017    209.700      

In [64]:
import coef_plots
import importlib
importlib.reload(coef_plots)
point_estimates = np.copy(res_punc.params.values)
# add intercept to male factor
point_estimates[1] = point_estimates[0]+point_estimates[1]
lowers = point_estimates - 1.96*res_punc.bse.values
uppers = point_estimates + 1.96*res_punc.bse.values
names = ["female", "male"]
colors = ["black", "blue"]
coef_plots.point_ests_and_CIs(point_estimates, lowers, uppers, names, colors, "punctual.pdf", xtitle="Punctuality" )

/anaconda/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [41]:
y_staff, X = dmatrices('y_staff ~ sex', data=dr_frame.dropna(subset=["y_staff"]), return_type='dataframe')
mod = sm.OLS(y_staff, X)
res_staff = mod.fit()
print(res_staff.summary())

                            OLS Regression Results                            
Dep. Variable:                y_staff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     29.07
Date:                Fri, 28 Oct 2016   Prob (F-statistic):           7.10e-08
Time:                        17:03:17   Log-Likelihood:                -24477.
No. Observations:               15290   AIC:                         4.896e+04
Df Residuals:                   15288   BIC:                         4.897e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       3.7263      0.018    204.292      

In [65]:
import coef_plots
import importlib
importlib.reload(coef_plots)
point_estimates = np.copy(res_staff.params.values)
# add intercept to male factor
point_estimates[1] = point_estimates[0]+point_estimates[1]
lowers = point_estimates - 1.96*res_staff.bse.values
uppers = point_estimates + 1.96*res_staff.bse.values
names = ["female", "male"]
colors = ["black", "blue"]
coef_plots.point_ests_and_CIs(point_estimates, lowers, uppers, names, colors, "staff.pdf", xtitle="Staff" )

/anaconda/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


## Predictive analysis; I think Michael's is much better!

In [ ]:
vectorizer = CountVectorizer(max_features=20000, min_df=25, ngram_range=(1,1), binary=True, stop_words=stop_words)
X = vectorizer.fit_transform(texts)
y = sexes
param_grid = {"C":[.01, .1, 1]}
clf = GridSearchCV(LogisticRegression(), param_grid=param_grid, scoring="f1_macro")

In [ ]:
clf.fit(X,y)

In [ ]:
clf.best_score_

In [ ]:
def show_most_informative_features(vectorizer, clf, n=50):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [ ]:
show_most_informative_features(vectorizer, clf.best_estimator_, n=100)